In [125]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange, tqdm, tqdm_notebook
from tensorboardX import SummaryWriter
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [139]:
def makedata(n):
    x = np.random.random((500, 2)) * 100
    mask = x.sum(axis=1) > x.mean()
    sums = x.sum(axis=1)
    sum2 = (x).prod(axis=1)

    y = sums*mask + sum2*(~mask)
    y = y.reshape((-1, 1))
    meanmark = x.mean()
    return x, y, meanmark

x, y, meanmark = makedata(500)
dx, dy, _ = makedata(1000)

class MLP(nn.Module):
    def __init__(self, idim, hdim, odim):
        super().__init__()
        self.idim = idim
        self.hdim = hdim
        self.odim = odim
        
        self.ilay = nn.Linear(idim, hdim)
        self.hlay = nn.Linear(hdim, hdim)
        self.olay = nn.Linear(hdim, odim)
    
    def forward(self, inp):
        h = F.elu(self.ilay(inp))
        h = F.elu(self.hlay(h))
        o = F.elu(self.olay(h))
        return o

class SplitNet(nn.Module):
    def __init__(self, idim, hdim, odim):
        super().__init__()
        self.idim = idim
        self.odim = odim
        self.hdim = hdim
        self.inet = MLP(idim, hdim, hdim)
        
        self.addnet = MLP(hdim, hdim, 1)
        self.mulnet1 = MLP(hdim, hdim, hdim)
        self.mulnet2 = MLP(hdim, hdim, 1)
    
    def forward(self, inp):
        h = self.inet(inp)
        flag = ((h.sum()).data < meanmark).numpy()
        if flag:
            o = self.addnet(h)
        else:
            o = self.mulnet1(h)
            o = self.mulnet2(o)
        return o

net = SplitNet(2, 30, 1)

A = Variable(torch.from_numpy(x).float())
B = Variable(torch.from_numpy(y).float())
C = Variable(torch.from_numpy(dx).float())
D = Variable(torch.from_numpy(dy).float())


criterion = nn.MSELoss()
opt = torch.optim.Adam(net.parameters())
!rm -rf logs
twriter = SummaryWriter('logs/train/')
dwriter = SummaryWriter('logs/dev/')
for epoch in trange(10000):
    l = 0
    for i, j in zip(A, B):
        net.zero_grad()
        p = net.forward(i)
        loss = criterion(p, j)
        loss.backward()
        l += (loss.data[0])
        opt.step()
    twriter.add_scalar('mse', l / x.shape[0], epoch)
    l = 0
    for i, j in zip(C, D):
        p = net.forward(i)
        loss = criterion(p, j)
        l += (loss.data[0])
    dwriter.add_scalar('mse', l/dx.shape[0], epoch)



  0%|          | 12/10000 [00:08<1:52:06,  1.48it/s]
Exception in thread Thread-38:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/arjoonn/.local/share/virtualenvs/narrativeqa-YhfIL57q/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/home/arjoonn/.local/share/virtualenvs/narrativeqa-YhfIL57q/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 10%|█         | 1004/10000 [11:43<1:45:06,  1.43it/s]


KeyboardInterrupt: 

In [ ]:
plt.plot(log)